# 필수 라이브러리

In [25]:
import sys
sys.path.append('/project/kcure-08/lib/python3.7/site-packages')

import warnings
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

warnings.filterwarnings('ignore')

In [26]:
print(os.getcwd()) # 현재 경로
# print(os.listdir(os.getcwd())) # 현재 경로 파일

os.chdir('/project/kcure-08/src/1. Data/1. Colon') # 현재 경로 변경(데이터 파일)
print(os.getcwd())
# print(os.listdir(os.getcwd()))

/project/kcure-08/src/1. Data/1. Colon
/project/kcure-08/src/1. Data/1. Colon


In [27]:
pd.set_option('display.max_columns', 150)
pd.set_option('display.max_rows', 150)

# 1. 데이터 불러오기

In [45]:
df_cc_rgst = pd.read_csv('./PP_csv/cs_cc_rgst.csv') # 21773 rows × 36 columns
df_cs_cc_rec = pd.read_csv('./PP_csv/cs_cc_rec_2.csv', index_col = 0)
df_cs_cc_nrec = pd.read_csv('./PP_csv/cs_cc_nrec_2.csv', index_col = 0)
df_cc_rgst # 21773 rows × 36 columns

In [ ]:
### RGST 중복 데이터 제거
df_cs_cc_rec = df_cs_cc_rec.drop(['SEX', 'FDX'], axis = 1)
df_cs_cc_rec

In [ ]:
### RGST 중복 데이터 제거
df_cs_cc_nrec = df_cs_cc_nrec.drop(['SEX', 'FDX'], axis = 1)
df_cs_cc_nrec

---

In [ ]:
### 불필요한 변수 한번에 제거
columns_to_drop = ['EXTEVAL', 'NODES', 'NODESEVAL', 'LNEXAM', 'METSEVAL','SSF2','SSF3' ,'SSF4', 'SSF5', 'SSF6', 'SSF7', 'SSF10', 'DREGDATE']

df_cc_rgst = df_cc_rgst.drop(columns = columns_to_drop)
df_cc_rgst

In [39]:
def custom_label_encoding(df, auto_encode_cols, manual_mapping):
    label_encoder = LabelEncoder()
    
    for column in auto_encode_cols:

        if column in auto_encode_cols:
            df[column] = label_encoder.fit_transform(df[column])
            
        elif column in manual_mapping:
            df[column] = df[column].map(manual_mapping[column])
    
    return df

auto_encode_cols = ['TCODE', 'MCODE_GRP', 'GRADE', 'AJCC6_T', 'AJCC6_TEVAL', 'AJCC6_N', 'AJCC6_NEVAL', 'AJCC6_M', 'AJCC6_MEVAL', 'AJCC6_STAGE', 'SS2000'] # 원발부위,조직학적진단코드, 분화도
manual_mapping = {}

In [40]:
df_cc_rgst_pp = custom_label_encoding(df_cc_rgst, auto_encode_cols, manual_mapping)

In [57]:
# T_SIZE # 000 - 989 까지 값, 990 - 999 nan값
t_size_col_pp = np.where(df_cc_rgst['T_SIZE'] <= 989, df_cc_rgst['T_SIZE'], np.nan)
df_cc_rgst_pp['T_SIZE'] = t_size_col_pp

In [58]:
# EXT # 000 - 900 까지 값, 999 nan값
ext_col_pp = np.where(df_cc_rgst['EXT'] <= 900, df_cc_rgst['EXT'], np.nan)
df_cc_rgst_pp['EXT'] = ext_col_pp

In [59]:
# LNPOS # 000 - 900 까지 값, 999 nan값
lmpos_col_pp = np.where(df_cc_rgst['LNPOS'] == 0, 0,
                        np.where((df_cc_rgst['LNPOS'] >= 1) & (df_cc_rgst['LNPOS'] <= 97), 1, np.nan))

df_cc_rgst_pp['LNPOS'] = lmpos_col_pp

In [60]:
# METS
mets_col_pp = np.where(df_cc_rgst['METS'] == 0, 0,
                        np.where((df_cc_rgst['METS'] >= 1) & (df_cc_rgst['METS'] <= 90), 1, np.nan))

df_cc_rgst_pp['METS'] = mets_col_pp

In [61]:
### SSF1 1: 양성 0: 음성(normal)
ssf1_col_pp = np.where(df_cc_rgst['SSF1'] == 10, 1,
                        np.where((df_cc_rgst['SSF1'] >= 20) & (df_cc_rgst['SSF1'] <= 30), 0, np.nan))

df_cc_rgst_pp['SSF1'] = ssf1_col_pp

In [62]:
### SSF2 코드명 정보와 불일치
### SSF3 코드명 정보와 불일치
### SSf6 코드명 정보와 불일치

In [63]:
### SSF8
ssf8_col_pp = np.where(df_cc_rgst['SSF8'] == 0, 0,
                       np.where(df_cc_rgst['SSF8'] == 10, 1,
                                np.where(df_cc_rgst['SSF8'] == 20, 2,
                                         np.where(df_cc_rgst['SSF8'] == 30, 3, np.nan))))

df_cc_rgst_pp['SSF8'] = ssf8_col_pp

In [64]:
### SSF9
ssf9_col_pp = np.where(df_cc_rgst['SSF9'] == 10, 1, 
                       np.where((df_cc_rgst['SSF9'] >= 20) & (df_cc_rgst['SSF9'] <= 30), 0, np.nan))

df_cc_rgst_pp['SSF9'] = ssf9_col_pp

In [65]:
# AJCC6_T (T1:0, T2:1, T3: 2, T4: 3, TX: np.nan) 
df_cc_rgst_pp['AJCC6_T'] = np.where(df_cc_rgst_pp['AJCC6_T'] == 4, np.nan, df_cc_rgst_pp['AJCC6_T'])

# AJCC6_TEVAL (c:0, p:1, yp: 2) 

# AJCC6_N (N1:0, N1:1, N2: 2, NX:np.nan) 
df_cc_rgst_pp['AJCC6_N'] = np.where(df_cc_rgst_pp['AJCC6_N'] == 3, np.nan, df_cc_rgst_pp['AJCC6_N'])

# AJCC6_NEVAL (c:0, p:1, yp: 2) 

# AJCC6_M (N0:0, N1:1, MX: np.nan) 
df_cc_rgst_pp['AJCC6_M'] = np.where(df_cc_rgst_pp['AJCC6_M'] == 2, np.nan, df_cc_rgst_pp['AJCC6_M'])

# AJCC6_MEVAL (c:0, p:1, yp: 2) 

# SS2000  (D(Distant : 0), L(Localized : 1), RE(Regional, direct extension: 2), RE+RN(Regional,extension and nodes: 3), RN(Regional, lymph nodes: 4), Is는 없없음. U:(unknown: 5) )
df_cc_rgst_pp['SS2000'] = np.where(df_cc_rgst_pp['SS2000'] == 5, np.nan, df_cc_rgst_pp['SS2000'])


---

In [23]:
col = ['SN_KEY','SEX', 'FDX', 'TCODE', 'MCODE_GRP', 'GRADE', 'AGE',
       'T_SIZE', 'EXT', 'LNPOS', 'METS', 'SSF1', 'SSF8', 'SSF9', 'AJCC6_T',
       'AJCC6_TEVAL', 'AJCC6_N', 'AJCC6_NEVAL', 'AJCC6_M', 'AJCC6_MEVAL',
       'AJCC6_STAGE', 'SS2000']

df_cc_rgst_pp = df_cc_rgst_pp[col]
df_cc_rgst_pp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21773 entries, 0 to 21772
Data columns (total 22 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   SN_KEY       21773 non-null  object 
 1   SEX          21773 non-null  int64  
 2   FDX          21773 non-null  int64  
 3   TCODE        21773 non-null  int64  
 4   MCODE_GRP    21773 non-null  int64  
 5   GRADE        21773 non-null  int64  
 6   AGE          21773 non-null  int64  
 7   T_SIZE       18403 non-null  float64
 8   EXT          20495 non-null  float64
 9   LNPOS        16131 non-null  float64
 10  METS         21317 non-null  float64
 11  SSF1         18726 non-null  float64
 12  SSF8         17697 non-null  float64
 13  SSF9         9609 non-null   float64
 14  AJCC6_T      20495 non-null  float64
 15  AJCC6_TEVAL  21773 non-null  int64  
 16  AJCC6_N      20809 non-null  float64
 17  AJCC6_NEVAL  21773 non-null  int64  
 18  AJCC6_M      21317 non-null  float64
 19  AJCC

-----

##### Label 1 : 재발

In [ ]:
## 재발피험자 불러오기 ###
cust_id = list(df_cs_cc_rec['SN_KEY'])
df_cc_rgst_rec = df_cc_rgst_pp[df_cc_rgst_pp['SN_KEY'].isin(cust_id)]
df_cc_rgst_rec

In [ ]:
df_cs_cc_rec = pd.merge(df_cs_cc_rec, df_cc_rgst_rec, how = 'inner', on =['SN_KEY'])
df_cs_cc_rec

In [34]:
df_cs_cc_rec.to_csv('./PP_csv/cs_cc_rec_3.csv', index = False)

----

##### Label 0 : 재발 x

In [ ]:
## 재발이 아닌 피험자 불러오기 ###
cust_id = list(df_cs_cc_nrec['SN_KEY'])
df_cc_rgst_nrec = df_cc_rgst_pp[df_cc_rgst_pp['SN_KEY'].isin(cust_id)]
df_cc_rgst_nrec = df_cc_rgst_nrec.drop_duplicates(subset='SN_KEY')
df_cc_rgst_nrec

In [ ]:
df_cs_cc_nrec = pd.merge(df_cs_cc_nrec, df_cc_rgst_nrec, how = 'inner', on =['SN_KEY'])
df_cs_cc_nrec

In [40]:
df_cs_cc_nrec.to_csv('./PP_csv/cs_cc_nrec_3.csv', index = False)